In [12]:
# Import Libraries
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns

# Database connection
# Connect to psycopg2 database (or create it if it doesn't exist)
from sqlalchemy import create_engine

connection = psycopg2.connect("dbname=Teleco user=postgres password=Leul@123")

# Create a cursor object to interact with the database
cursor = connection.cursor()

# Set plot style
sns.set(style="whitegrid")

# Step 1: Retrieve and Document Top Handsets and Manufacturers
query_handsets = """
SELECT manufacturer, handset, COUNT(*) AS usage_count
FROM xdr_data
GROUP BY manufacturer, handset
ORDER BY usage_count DESC
LIMIT 10;
"""
df_handsets = pd.read_sql_query(query_handsets, connection)

# Generate Bar Plot for Top Handsets
plt.figure(figsize=(12, 6))
sns.barplot(x='handset', y='usage_count', data=df_handsets, palette='viridis')
plt.title("Top 10 Handsets by Usage", fontsize=16)
plt.xlabel("Handset", fontsize=12)
plt.ylabel("Usage Count", fontsize=12)
plt.xticks(rotation=45)
plt.savefig("top_handsets.png")  # Save the figure
plt.show()

# Step 2: Aggregate Metrics
query_metrics = """
SELECT
    SUM(sessions) AS total_sessions,
    AVG(duration) AS avg_duration,
    SUM(download) AS total_download,
    SUM(upload) AS total_upload
FROM usage_table;
"""
df_metrics = pd.read_sql_query(query_metrics, conn)

# Documenting Insights
print("=== Aggregate Metrics ===")
print(f"Total Sessions: {df_metrics['total_sessions'][0]:,}")
print(f"Average Session Duration: {df_metrics['avg_duration'][0]:.2f}")
print(f"Total Download: {df_metrics['total_download'][0]:,} MB")
print(f"Total Upload: {df_metrics['total_upload'][0]:,} MB")

# Step 3: Session Duration Decile Analysis
query_durations = "SELECT duration FROM usage_table;"
df_durations = pd.read_sql_query(query_durations, conn)

# Calculate Deciles
df_durations['decile'] = pd.qcut(df_durations['duration'], 10, labels=False)
decile_counts = df_durations['decile'].value_counts().sort_index()

# Plot Decile Analysis
plt.figure(figsize=(10, 5))
sns.barplot(x=decile_counts.index, y=decile_counts.values, palette="coolwarm")
plt.title("Session Duration by Decile", fontsize=16)
plt.xlabel("Decile", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.savefig("session_decile_analysis.png")  # Save the figure
plt.show()

# Step 4: Automating Report Generation
report_content = f"""
# User Overview Analysis

## Top 10 Handsets
Top handset is **{df_handsets.iloc[0]['handset']}**, which accounts for **{df_handsets.iloc[0]['usage_count']}** uses.

## Aggregate Metrics
- **Total Sessions**: {df_metrics['total_sessions'][0]:,}
- **Average Session Duration**: {df_metrics['avg_duration'][0]:.2f} seconds
- **Total Download**: {df_metrics['total_download'][0]:,} MB
- **Total Upload**: {df_metrics['total_upload'][0]:,} MB

## Visualizations
### 1. Top 10 Handsets Usage
![Top Handsets](top_handsets.png)

### 2. Session Duration Decile Analysis
![Session Deciles](session_decile_analysis.png)
"""

with open("report.md", "w") as file:
    file.write(report_content)

print("Report saved as 'report.md'. View it for insights and graphs.")


C:\Users\Almazt\AppData\Local\Temp\ipykernel_9932\746586722.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_handsets = pd.read_sql_query(query_handsets, connection)


DatabaseError: Execution failed on sql '
SELECT manufacturer, handset, COUNT(*) AS usage_count
FROM xdr_data
GROUP BY manufacturer, handset
ORDER BY usage_count DESC
LIMIT 10;
': column "manufacturer" does not exist
LINE 2: SELECT manufacturer, handset, COUNT(*) AS usage_count
               ^
